In [1]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [2]:
class Person:
    def __init__(self,name):
        self.name = name
        
    def __repr__(self):
        return f'Person({self.name})'
    
    def __del__(self):
        print(f'__del__ calledd for {self}...')

In [3]:
p = Person('Dean')

In [4]:
id_p = id(p)
ref_count(id_p)

1

In [5]:
p = None

__del__ calledd for Person(Dean)...


In [6]:
p = Person('Sam')

In [7]:
del p

__del__ calledd for Person(Sam)...


In [9]:
class Person:
    def __init__(self,name):
        self.name = name
        
    def __repr__(self):
        return f'Person({self.name})'
    
    def __del__(self):
        print(f'__del__ calledd for {self}...')
        
    def gen_ex(self):
        raise ValueError('Something went bump...')

In [11]:
p = Person('Eric')
p_id = id(p)
ref_count(p_id)

__del__ calledd for Person(Eric)...


1

In [13]:
try:
    p.gen_ex()
except ValueError as ex:
    error =ex
    print(ex)

Something went bump...


In [14]:
error

ValueError('Something went bump...')

In [15]:
ref_count(p_id)

2

In [16]:
dir(error)

['__cause__',
 '__class__',
 '__context__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__suppress_context__',
 '__traceback__',
 'args',
 'with_traceback']

In [17]:
dir(error.__traceback__)

['tb_frame', 'tb_lasti', 'tb_lineno', 'tb_next']

In [19]:
dir(error.__traceback__.tb_frame)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'f_back',
 'f_builtins',
 'f_code',
 'f_globals',
 'f_lasti',
 'f_lineno',
 'f_locals',
 'f_trace']

In [20]:
type(error.__traceback___.tb_frame.f_locals)

AttributeError: 'ValueError' object has no attribute '__traceback___'

In [25]:
for key,value in error.__traceback__.tb_frame.f_locals.copy().items():
    if isinstance(value,Person):
        print(key,value,id(value))

p Person(Eric) 4577871240


In [26]:
ref_count(p_id)

2

In [27]:
del p 

In [28]:
del error

In [29]:
ref_count(p_id)

1

In [30]:
class Person:
    def __del__(self):
        raise ValueError('Something went bump...')

In [31]:
p = Person()
del p
print('All good, no exceptions interrupted our code')

All good, no exceptions interrupted our code


Exception ignored in: <bound method Person.__del__ of <__main__.Person object at 0x111e8d2e8>>
Traceback (most recent call last):
  File "<ipython-input-30-6ed6e62e38b8>", line 3, in __del__
ValueError: Something went bump...


In [32]:
import sys
sys.stderr,sys.stdout

(<ipykernel.iostream.OutStream at 0x110be7b38>,
 <ipykernel.iostream.OutStream at 0x110be7b70>)

In [45]:
class ErrToFile:
    def __init__(self,fname):
        self._fname = fname
        self._current_stderr = sys.stderr
        
    def __enter__(self):
        self._file = open(self._fname,'w')
        sys.stderr = self._file
    
    def __exit__(self,exc_type,exc_value,exc_tb):
        sys.stderr = self._current_stderr
        if self._file:
            self._file.close()
        return False
    
    

In [46]:
p = Person()

In [47]:
with ErrToFile('err.txt'):
    del p
    print(100)
print(200)
print(300)

100
200
300
